In [1]:
output_path = "./crop_output"

In [2]:
import os
import cv2
import numpy as np
from patchify import patchify
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
import tifffile as tiff

2024-05-22 19:49:40.321554: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-22 19:49:40.505732: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-22 19:49:40.505762: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-22 19:49:40.507616: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-22 19:49:40.617867: I tensorflow/core/platform/cpu_feature_g

In [3]:
number_of_mask = 0
mask_train_dataset = []
mask_val_dataset = []
for folder_name in sorted(os.listdir(output_path)):  # Ensure folders are iterated in sorted order
    folder_path = os.path.join(output_path, folder_name)
    if os.path.isdir(folder_path) and folder_name=='train':
        for image_folder_name in sorted(os.listdir(folder_path)):  # Sort subfolders as well
            image_folder_path = os.path.join(folder_path, image_folder_name)
            # print(image_folder_path)
            if 'combined_mask' in image_folder_path:
                for image_name in sorted(os.listdir(image_folder_path)):  # Sort image filenames
                    image_path = os.path.join(image_folder_path, image_name)
                    img = cv2.imread(image_path, 1)
                    mask_train_dataset.append(img)
                    number_of_mask += 1

    if os.path.isdir(folder_path) and folder_name=='val':
        for image_folder_name in sorted(os.listdir(folder_path)):  # Sort subfolders as well
            image_folder_path = os.path.join(folder_path, image_folder_name)
            # print(image_folder_path)
            if 'combined_mask' in image_folder_path:
                for image_name in sorted(os.listdir(image_folder_path)):  # Sort image filenames
                    image_path = os.path.join(image_folder_path, image_name)
                    img = cv2.imread(image_path, 1)
                    mask_val_dataset.append(img)
                    number_of_mask += 1

print(number_of_mask)

36349


In [4]:
mask_train_dataset = np.array(mask_train_dataset)
mask_val_dataset = np.array(mask_val_dataset)

In [5]:
mask_train_dataset = np.mean(mask_train_dataset, axis=-1, dtype=np.uint8, keepdims=True)

In [6]:
print(mask_train_dataset.shape)

(31154, 256, 256, 1)


In [7]:
mask_val_dataset = np.mean(mask_val_dataset, axis=-1, dtype=np.uint8, keepdims=True)

In [8]:
print(mask_val_dataset.shape)

(5195, 256, 256, 1)


In [9]:
save_train_mask_image_dataset_path = './save_data_path/mask_train_image_dataset.npy'
save_val_mask_image_dataset_path = './save_data_path/mask_val_image_dataset.npy'

save_mask_chunk1_path = 'save_data_path/mask_chunk1.npy'
save_mask_chunk2_path = 'save_data_path/mask_chunk2.npy'
save_mask_chunk3_path = 'save_data_path/mask_chunk3.npy'
save_mask_chunk4_path = 'save_data_path/mask_chunk4.npy'
save_mask_chunk5_path = 'save_data_path/mask_chunk5.npy'
save_mask_chunk6_path = 'save_data_path/mask_chunk6.npy'

In [10]:
os.makedirs('./save_data_path', exist_ok=True)

In [11]:
temp1 = to_categorical(mask_train_dataset[0:5000], dtype="uint8")
np.save(save_mask_chunk1_path, temp1)
del temp1

temp2 = to_categorical(mask_train_dataset[5000:10000], dtype="uint8")
np.save(save_mask_chunk2_path, temp2)
del temp2

temp3 = to_categorical(mask_train_dataset[10000:15000], dtype="uint8")
np.save(save_mask_chunk3_path, temp3)
del temp3

temp4 = to_categorical(mask_train_dataset[15000:20000], dtype="uint8")
np.save(save_mask_chunk4_path, temp4)
del temp4

temp5 = to_categorical(mask_train_dataset[20000:25000], dtype="uint8")
np.save(save_mask_chunk5_path, temp5)
del temp5

temp6 = to_categorical(mask_train_dataset[25000:], dtype="uint8")
np.save(save_mask_chunk6_path, temp6)
del temp6

temp1 = np.load(save_mask_chunk1_path, mmap_mode='c')
temp2 = np.load(save_mask_chunk2_path, mmap_mode='c')
temp3 = np.load(save_mask_chunk3_path, mmap_mode='c')
temp4 = np.load(save_mask_chunk4_path, mmap_mode='c')
temp5 = np.load(save_mask_chunk5_path, mmap_mode='c')
temp6 = np.load(save_mask_chunk6_path, mmap_mode='c')

mask_train_dataset = np.concatenate((temp1, temp2, temp3, temp4, temp5, temp6), axis=0)
np.save(save_train_mask_image_dataset_path, mask_train_dataset)

print(mask_train_dataset.shape)

del mask_train_dataset

: 

In [ ]:
mask_val_dataset = to_categorical(mask_val_dataset, dtype="uint8")

np.save(save_val_mask_image_dataset_path, mask_val_dataset)

del mask_val_dataset